In [4]:
import pandas as pd
import requests
import json
import re
import os
from dotenv import load_dotenv

load_dotenv()

In [3]:
class openai:
    def __init__(self) -> None:
        pass

    def consumer_details(self):
        try:
            """get consumer details"""
            url = os.getenv("vapt_api") 
            response = requests.get(url )

            if response.status_code == 200:
                print("consumer details success")
                return response.json() 
        except Exception as e:
            print("consumer details error:", e)

    def get_population(self,year):
        """get usa population"""
        url = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
        response = requests.get(url )
    
        if response.status_code == 200:
            # print("Success:", response.json())
            print("usa popluation data success")
            return response.json()
        else:
            print("Error in api:", response.status_code, response.text)
    
    def get_current_weather(self, location, unit="celcius"):
        """Get the current weather in a given location"""
        if "tokyo" in location.lower():
            return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
        elif "san francisco" in location.lower():
            return json.dumps({"location": "San Francisco", "temperature": "32", "unit": unit})
        elif "paris" in location.lower():
            return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
        else:
            return json.dumps({"location": location, "temperature": "unknown"})
        
    def format_currency_as_rupees(self,response):
        # Use regex to find and replace dollar amounts with rupees
        return re.sub(r'\$\s*([\d,]+(?:\.\d+)?)', r'₹\1', response)